In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications import MobileNetV2
from keras.preprocessing import image

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
#Create Train Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/gdrive/MyDrive/data',
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=100,
    image_size=(224,224),
    validation_split=0.2,
    seed=123,
    subset='training')

In [ ]:
#Create Test Dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/gdrive/MyDrive/data',
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=100,
    image_size=(224,224),
    validation_split=0.2,
    seed=123,
    subset='validation')

In [ ]:
import matplotlib.image as mpimg
import cv2

directory = os.listdir('/content/gdrive/MyDrive/data')
for each in directory:
  plt.figure()
  currentFolder = '/content/gdrive/MyDrive/data/' + each
  for i, file in enumerate(os.listdir(currentFolder)[:10]):
    fullpath = currentFolder + '/' + file
    img = mpimg.imread(fullpath)
    plt.subplot(2,5,i+1)
    plt.title(fullpath.split('/')[5])
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
#Inception V3
inceptionv3 = InceptionV3(input_shape = (224,224,3), #shape of resized image
                                include_top = False,
                                weights = None)

In [ ]:
x = inceptionv3.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
#Create Model 1
model1 = Model(inceptionv3.input, outputs=predictions)
model1.compile(optimizer='Adam', loss='categorical_crossentropy',
               metrics=['acc',
                        tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall(),
                        tf.keras.metrics.AUC()]
               )

In [ ]:
#Show Layers for Model 1
for i, layer in enumerate(inceptionv3.layers):
   print(i, layer.name)

In [ ]:
#Fit Model 1
history = model1.fit(
            train_ds,
            validation_data = test_ds,
            steps_per_epoch = 5,
            epochs = 5,
            validation_steps = 5,
            verbose = 1)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#summarize history for auc
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('model auc')
plt.ylabel('auc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#Show metrics for Model 1
precision = history.history['precision'][4]
recall = history.history['recall'][4]
f1_score = (2*precision*recall)/(precision + recall)
print('precision:',precision)
print('recall:',recall)
print('F1 score:',f1_score)

In [ ]:
#ResNet50
resnet50 = ResNet50(input_shape = (224,224,3), #shape of resized image
                                include_top = False,
                                weights = None)

In [ ]:
x = resnet50.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
#Model 2
model2 = Model(inputs=resnet50.input, outputs=predictions)
model2.compile(optimizer='Adam', loss='binary_crossentropy', 
               metrics=['acc',
                        tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall(),
                        tf.keras.metrics.AUC()])

In [ ]:
#Show Layers for model 2
for i, layer in enumerate(resnet50.layers):
   print(i, layer.name)

In [ ]:
#Fit Model 2
history2 = model2.fit(
            train_ds,
            validation_data = test_ds,
            steps_per_epoch = 5,
            epochs = 5,
            validation_steps = 5,
            verbose = 1)

In [ ]:
# summarize history for accuracy
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#Show metrics for Model 2
precision = history2.history['precision_1'][4]
recall = history2.history['recall_1'][4]
f1_score = (2*precision*recall)/(precision + recall)
print('precision:',precision)
print('recall:',recall)
print('F1 score:',f1_score)

In [ ]:
# model3 = MobileNetV2
mobilenetv2 = MobileNetV2(input_shape = (224,224,3), #shape of resized image
                                include_top = False,
                                weights = None)

In [ ]:
x=mobilenetv2.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(2,activation='softmax')(x) 

model3=Model(inputs=mobilenetv2.input,outputs=preds)

In [ ]:
#Model 3
model3.compile(optimizer='Adam',loss='categorical_crossentropy',
               metrics=['acc',
                        tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall(),
                        tf.keras.metrics.AUC(),
                        tf.keras.metrics.TruePositives(),
                        tf.keras.metrics.TrueNegatives(),
                        tf.keras.metrics.FalsePositives(),
                        tf.keras.metrics.FalseNegatives()])

In [ ]:
#Show Layers for Model 3
for i, layer in enumerate(mobilenetv2.layers):
   print(i, layer.name)

In [ ]:
#Fit Model 3
history3 = model3.fit(
            train_ds,
            validation_data = test_ds,
            steps_per_epoch = 5,
            epochs = 5,
            validation_steps = 5,
            verbose = 1)

In [ ]:
print(history3.history['true_positives'])
tpr = []
fpr = []
for i in range(5):
  tpr.append((history3.history['true_positives'][i])/(history3.history['true_positives'][i]+history3.history['false_negatives'][i]))
  fpr.append((history3.history['false_positives'][i])/(history3.history['false_positives'][i]+history3.history['true_negatives'][i]))
plt.plot(tpr,fpr)

In [ ]:
# summarize history for accuracy
plt.plot(history3.history['acc'])
plt.plot(history3.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#Show metrics for Model 3
precision = history3.history['precision_2'][4]
recall = history3.history['recall_2'][4]
f1_score = (2*precision*recall)/(precision + recall)
print('precision:',precision)
print('recall:',recall)
print('F1 score:',f1_score)

In [ ]:
#Predicting Negative Image
models=[]
models.append(model1)
models.append(model2)
models.append(model3)
path = '/content/gdrive/MyDrive/data/negative/00a85be6-6eb0-421d-8acf-ff2dc0007e8a.png' 
img = image.load_img(path, target_size=(224, 224)) #path and size
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
labels = []
for m in models:
    predicts = np.argmax(m.predict(images), axis=1)
    labels.append(predicts)
    
# Ensemble with voting
labels = np.array(labels)
labels = np.transpose(labels, (1, 0))
labels = scipy.stats.mode(labels, axis=1)[0]
labels = np.squeeze(labels)
if labels == 0:
  print('Covid Negative')
else:
  print('Covid Positive')
plt.imshow(img)

In [ ]:
#Predicting Positive Image
path = '/content/gdrive/MyDrive/data/positive/0a6c60063b4bae4de001caaba306d1_jumbo.jpeg' 
img = image.load_img(path, target_size=(224, 224)) #path and size
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
labels = []
for m in models:
    predicts = np.argmax(m.predict(images), axis=1)
    labels.append(predicts)
    
# Ensemble with voting
labels = np.array(labels)
labels = np.transpose(labels, (1, 0))
labels = scipy.stats.mode(labels, axis=1)[0]
labels = np.squeeze(labels)
if labels == 0:
  print('Covid Negative')
else:
  print('Covid Positive')
plt.imshow(img)

In [ ]:
#Predicting Set of Images
paths = []

for filename in os.listdir('/content/gdrive/MyDrive/data/test'):
    if filename.endswith("jpg") or filename.endswith("png"): 
        paths.append(filename)
i = 0
for path in paths:
  img = image.load_img(path, target_size=(224, 224)) #path and size
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  labels = []
  for m in models:
      predicts = np.argmax(m.predict(images), axis=1)
      labels.append(predicts)
      
  # Ensemble with voting
  labels = np.array(labels)
  labels = np.transpose(labels, (1, 0))
  labels = scipy.stats.mode(labels, axis=1)[0]
  labels = np.squeeze(labels)
  if labels == 0:
    print('Covid Negative')
  else:
    print('Covid Positive')
  plt.subplot(2,5,i+1)
  plt.title(path.split('/')[5])
  plt.axis('off')
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  i = (i+1)%6

In [ ]:
directory = os.listdir('/content/gdrive/MyDrive/data')
test = []
x = 0
for each in directory:
  currentFolder = '/content/gdrive/MyDrive/data/' + each
  for i, file in enumerate(os.listdir(currentFolder)[:40]):
    fullpath = currentFolder + '/' + file
    test.append([fullpath,x])
  x=x+1

In [ ]:
import random
random.shuffle(test)

In [ ]:
y_label = []
for i,j in test:
  y_label.append(j)
print(y_label)

In [ ]:
models=[]
models.append(model1)
models.append(model2)
models.append(model3)

In [ ]:
def ensemble_test(path, models):
  img = image.load_img(path, target_size=(224, 224)) #path and size
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  labels = []
  for m in models:
      predicts = np.argmax(m.predict(images), axis=1)
      labels.append(predicts)
      
  # Ensemble with voting
  labels = np.array(labels)
  labels = np.transpose(labels, (1, 0))
  labels = scipy.stats.mode(labels, axis=1)[0]
  labels = np.squeeze(labels)
  return labels

In [ ]:
y_predict = []
for i,j in test:
  y_predict.append(ensemble_test(i, models))

In [ ]:
print(y_predict)

In [ ]:
def calc_metric(y_actual, y_hat):
    fpr = []
    tpr = []
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1
        if (FP+TN) != 0:
          x = FP/(FP+TN)
        else:
          x = 0
        if (TP+FN) != 0:
          y = TP/(TP+FN)
        else:
          y = 0 
        
        fpr.append(x)
        tpr.append(y)

    return(TP, FP, TN, FN, fpr, tpr)

In [ ]:
TP, FP, TN, FN, fpr, tpr = calc_metric(y_label, y_predict)

In [ ]:
correct_label = np.array(y_label) == np.array(y_predict)
print("Accuracy = %2.2f%%" % (np.mean(correct_label)*100))

In [ ]:
precision = TP / (TP + FP)
print("Precision: ", precision)

In [ ]:
import sklearn.metrics as metrics
confusion_matrix = metrics.confusion_matrix(y_true=y_label, y_pred=y_predict)
print(confusion_matrix)

In [ ]:
import seaborn as sn
import pandas as pd

df_cm = pd.DataFrame(confusion_matrix, index = ['True', 'False'],
                  columns = [i for i in ['Positive', 'Negative']])
plt.figure(figsize = (2,2))
sn.heatmap(df_cm, annot=True)

In [ ]:
plt.plot(fpr, tpr)
plt.title('ROC Curve')
plt.show() 

# This is the AUC
auc = np.trapz(tpr,fpr)